In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures

# Load stem point data from file
points = np.loadtxt('stem_points.txt')

# Check units and convert if necessary
z_values = points[:, 2]

# Detect if data is in meters vs centimeters
if z_values.max() < 50:  # Likely in meters
    # Convert to centimeters for analysis
    points_cm = points.copy()
    points_cm[:, 2] *= 100  # Convert Z to centimeters
    analysis_points = points_cm
    units = "cm"
else:  # Already in centimeters
    analysis_points = points
    units = "cm"

# Function to fit a circle using median radius
def robust_circle_fit(points_xy):
    x = points_xy[:, 0]
    y = points_xy[:, 1]
    x_mean, y_mean = np.mean(x), np.mean(y)
    x_cent = x - x_mean
    y_cent = y - y_mean
    r = np.sqrt(x_cent**2 + y_cent**2)
    radius = np.median(r)
    return x_mean, y_mean, radius

# Function to calculate cylinder axis orientation
def calculate_cylinder_axis(points_3d):
    """
    Calculate the central axis of a cylinder fitted to 3D points
    Returns the axis direction vector and deviation from vertical in degrees
    """
    if len(points_3d) < 10:  # Need sufficient points
        return None, 90.0  # Return high deviation if insufficient points
    
    # Fit a line through the 3D points using RANSAC for robustness
    X = points_3d[:, 2].reshape(-1, 1)  # Z coordinates (height)
    y_x = points_3d[:, 0]  # X coordinates
    y_y = points_3d[:, 1]  # Y coordinates
    
    try:
        # Fit line for X vs Z
        ransac_x = RANSACRegressor(random_state=42, min_samples=5)
        ransac_x.fit(X, y_x)
        slope_x = ransac_x.estimator_.coef_[0]
        
        # Fit line for Y vs Z
        ransac_y = RANSACRegressor(random_state=42, min_samples=5)
        ransac_y.fit(X, y_y)
        slope_y = ransac_y.estimator_.coef_[0]
        
        # Construct the direction vector of the cylinder axis
        # Direction vector: (slope_x, slope_y, 1) normalized
        axis_vector = np.array([slope_x, slope_y, 1])
        axis_vector = axis_vector / np.linalg.norm(axis_vector)
        
        # Vertical reference vector
        vertical_vector = np.array([0, 0, 1])
        
        # Calculate angle between axis and vertical
        dot_product = np.dot(axis_vector, vertical_vector)
        # Clamp dot product to avoid numerical errors
        dot_product = np.clip(dot_product, -1.0, 1.0)
        angle_rad = np.arccos(np.abs(dot_product))  # Use abs to get acute angle
        angle_deg = np.degrees(angle_rad)
        
        return axis_vector, angle_deg
        
    except Exception as e:
        print(f"Error calculating cylinder axis: {e}")
        return None, 90.0

# Function to process a bin and check for subdivision
def process_bin_adaptive(points, z_start, z_end, min_bin_size=15.0, max_deviation=2.0):
    """
    Process a bin and subdivide if cylinder axis is NOT vertical (deviation > 2°)
    Returns list of processed sub-bins with their data
    """
    bin_height = z_end - z_start
    
    # Extract points in this bin
    bin_points = points[(points[:, 2] >= z_start) & (points[:, 2] < z_end)]
    
    if len(bin_points) < 5:  # Insufficient points
        return []
    
    # Calculate cylinder axis orientation
    axis_vector, deviation_deg = calculate_cylinder_axis(bin_points)
    
    # Check if we should subdivide
    should_subdivide = (
        deviation_deg > max_deviation and  # NOT vertical (tilted)
        bin_height > min_bin_size  # Can still subdivide
    )
    
    if should_subdivide:
        # Subdivide into two halves
        z_mid = (z_start + z_end) / 2
        
        # Recursively process each half
        upper_bins = process_bin_adaptive(points, z_mid, z_end, min_bin_size, max_deviation)
        lower_bins = process_bin_adaptive(points, z_start, z_mid, min_bin_size, max_deviation)
        
        return lower_bins + upper_bins
    else:
        # Check if this is a final bin that should be ignored
        if bin_height <= min_bin_size and deviation_deg > max_deviation:
            # This is a 15cm bin that's still tilted - ignore it
            return [('IGNORED', z_start, z_end, bin_points, axis_vector, deviation_deg)]
        else:
            # Process this bin as final (either vertical or acceptable)
            return [(z_start, z_end, bin_points, axis_vector, deviation_deg)]

# Parameters
thickness = 0.15  # units: same as your point coordinates (likely cm)
initial_buffer_size_cm = 100  # Start with 1m bins
min_bin_size_cm = 15  # Minimum bin size (15cm)
max_deviation_deg = 2.0  # Maximum deviation from vertical (2 degrees)

min_height = analysis_points[:, 2].min()
max_height = analysis_points[:, 2].max()
total_height = max_height - min_height

colors_useful = ['orange', 'green', 'blue', 'purple', 'red', 'cyan', 'brown', 'pink', 'yellow', 'lime']
output_data = []

# Plot setup
fig = plt.figure(figsize=(16, 12))
ax = fig.add_subplot(111, projection='3d')
legend_patches = []

# Create initial 1m buffer ranges
initial_buffer_count = int(np.ceil(total_height / initial_buffer_size_cm))
all_processed_bins = []

print("Processing bins with adaptive subdivision based on cylinder axis verticality...")
print(f"Initial bin size: {initial_buffer_size_cm} cm")
print(f"Minimum bin size: {min_bin_size_cm} cm")
print(f"Subdivision rule: If deviation > {max_deviation_deg}° (tilted) AND bin > {min_bin_size_cm}cm → subdivide")
print(f"Ignore rule: If final bin is {min_bin_size_cm}cm and deviation > {max_deviation_deg}° → ignore")
print("-" * 60)

# Process each initial 1m bin
for i in range(initial_buffer_count):
    z_start = min_height + (i * initial_buffer_size_cm)
    z_end = min(z_start + initial_buffer_size_cm, max_height)
    
    print(f"Processing initial bin {i+1}: {z_start:.1f} - {z_end:.1f} cm")
    
    # Apply adaptive subdivision to this bin
    processed_bins = process_bin_adaptive(
        analysis_points, z_start, z_end, min_bin_size_cm, max_deviation_deg
    )
    
    print(f"  -> Subdivided into {len(processed_bins)} final bins")
    for j, (sub_start, sub_end, _, _, dev) in enumerate(processed_bins):
        print(f"     Sub-bin {j+1}: {sub_start:.1f}-{sub_end:.1f} cm, deviation: {dev:.2f}°")
    
    all_processed_bins.extend(processed_bins)

print(f"\nTotal final bins: {len(all_processed_bins)}")
print("=" * 60)

# Process and visualize each final bin
buffer_id = 1
ignored_bins = 0

for bin_data in all_processed_bins:
    # Check if this bin should be ignored
    if bin_data[0] == 'IGNORED':
        ignored_bins += 1
        print(f"Ignoring bin at {bin_data[1]:.1f}-{bin_data[2]:.1f} cm (15cm bin with {bin_data[5]:.2f}° deviation)")
        continue
        
    z_start, z_end, buffer_points, axis_vector, deviation_deg = bin_data
    
    if len(buffer_points) < 5:
        continue

    x_center, y_center, radius = robust_circle_fit(buffer_points[:, :2])
    diameter = 2 * radius
    height = z_end - z_start
    volume = (np.pi * radius**2 * height)
    surface_area = 2 * np.pi * radius * height + 2 * np.pi * radius**2
    surface_density = buffer_points.shape[0] / surface_area if surface_area > 0 else 0

    # Calculate distances from cylinder center
    distances = np.sqrt((buffer_points[:, 0] - x_center)**2 + (buffer_points[:, 1] - y_center)**2)
    
    # Identify waste points as those outside radius + thickness
    waste_points_mask = distances > (radius + thickness)
    waste_points_count = np.sum(waste_points_mask)
    
    # Points within the fitted cylinder (inside radius)
    inside_cylinder_mask = distances <= radius
    inside_points_count = np.sum(inside_cylinder_mask)
    
    # Points in the thickness buffer zone (between radius and radius + thickness)
    buffer_zone_mask = (distances > radius) & (distances <= radius + thickness)
    buffer_zone_points_count = np.sum(buffer_zone_mask)
    
    # Total valid points (inside cylinder + buffer zone)
    valid_points_count = inside_points_count + buffer_zone_points_count

    # Cylinder equation format
    cylinder_equation = f"(x - {x_center:.3f})² + (y - {y_center:.3f})² = {radius:.3f}²"

    # Calculate height from ground (for better interpretation)
    height_from_ground = z_start - min_height
    
    # Format axis vector for display
    axis_str = f"[{axis_vector[0]:.4f}, {axis_vector[1]:.4f}, {axis_vector[2]:.4f}]" if axis_vector is not None else "N/A"

    # Save cylinder data with axis information
    output_data.append([
        buffer_id, z_start, z_end, height_from_ground, diameter, radius, height,
        volume, surface_area, surface_density,
        inside_points_count, buffer_zone_points_count, waste_points_count, 
        valid_points_count, cylinder_equation, deviation_deg, axis_str
    ])

    # Cylinder mesh for visualization
    if radius > 0:
        theta = np.linspace(0, 2 * np.pi, 60)
        z = np.linspace(z_start, z_end, 10)
        theta_mesh, z_mesh = np.meshgrid(theta, z)

        # Color based on deviation - green for vertical, red for tilted
        if deviation_deg <= max_deviation_deg:
            color = (0, 0.8, 0, 0.4)  # Green for vertical
        else:
            color = (0.8, 0, 0, 0.4)  # Red for tilted
        
        # Outer cylinder (fit + thickness)
        x_outer = x_center + (radius + thickness) * np.cos(theta_mesh)
        y_outer = y_center + (radius + thickness) * np.sin(theta_mesh)
        ax.plot_surface(x_outer, y_outer, z_mesh, color=color, alpha=0.3, edgecolor='none')

        # Inner cylinder (fit)
        x_inner = x_center + radius * np.cos(theta_mesh)
        y_inner = y_center + radius * np.sin(theta_mesh)
        ax.plot_surface(x_inner, y_inner, z_mesh, color='gray', alpha=0.15, edgecolor='none')

        # Legend entry
        deviation_status = "Vertical" if deviation_deg <= max_deviation_deg else "Tilted"
        legend_patches.append(Patch(
            color=color[:3], alpha=0.4,
            label=f'Bin {buffer_id} ({height_from_ground:.0f}cm) - {deviation_status} ({deviation_deg:.1f}°)'
        ))

    buffer_id += 1

# Plot settings
ax.set_xlabel(f'X Axis ({units})')
ax.set_ylabel(f'Y Axis ({units})')
ax.set_zlabel(f'Height Z ({units})')
ax.set_title(f"Adaptive Stem Analysis - Subdivide Tilted Bins\nTotal Height: {total_height:.1f} {units} | Processed Bins: {len(output_data)} | Ignored: {ignored_bins}")

# Add legend (limit to prevent overcrowding)
if len(legend_patches) <= 15:
    ax.legend(handles=legend_patches, loc='center left', bbox_to_anchor=(1.05, 0.5), fontsize=8)
else:
    ax.legend(handles=legend_patches[:15], loc='center left', bbox_to_anchor=(1.05, 0.5), fontsize=8)
    print(f"Note: Legend limited to first 15 bins for clarity (total: {len(legend_patches)})")

plt.tight_layout()
plt.show()

# Save detailed analysis report
with open("adaptive_stem_analysis1.txt", "w", encoding="utf-8") as f:
    f.write("ADAPTIVE STEM CYLINDER ANALYSIS - SUBDIVIDE TILTED BINS\n")
    f.write("="*80 + "\n")
    f.write(f"Analysis Method: Subdivide bins when cylinder axis deviation > {max_deviation_deg}°\n")
    f.write(f"Initial bin size: {initial_buffer_size_cm} cm (1 meter)\n")
    f.write(f"Minimum bin size: {min_bin_size_cm} cm\n")
    f.write(f"Verticality threshold: {max_deviation_deg} degrees from vertical\n")
    f.write(f"Subdivision rule: If deviation > {max_deviation_deg}° and bin > {min_bin_size_cm}cm, divide in half\n")
    f.write(f"Ignore rule: If final bin is {min_bin_size_cm}cm and deviation > {max_deviation_deg}°, ignore it\n")
    f.write(f"Original data units: {'Meters' if z_values.max() < 50 else 'Centimeters'}\n")
    f.write(f"Analysis performed in: Centimeters\n")
    f.write(f"Tree base height: {min_height:.1f} cm\n")
    f.write(f"Tree top height: {max_height:.1f} cm\n")
    f.write(f"Total stem height: {total_height:.1f} cm ({total_height/100:.2f} m)\n")
    f.write(f"Total final bins processed: {len(output_data)}\n")
    f.write(f"Total bins ignored: {ignored_bins}\n")
    f.write(f"Total points: {len(analysis_points)}\n")
    f.write("="*80 + "\n")
    f.write("POINT CLASSIFICATION:\n")
    f.write("- Inside Cylinder: Points within fitted radius\n")
    f.write("- Buffer Zone: Points between radius and radius + thickness\n")
    f.write("- Waste Points: Points beyond radius + thickness\n")
    f.write("="*80 + "\n")
    f.write("AXIS ORIENTATION:\n")
    f.write("- Cylinder axis calculated using RANSAC line fitting through 3D points\n")
    f.write("- Deviation measured as angle between axis and vertical (Z) direction\n")
    f.write("- Bins with deviation > 2° are subdivided if bin size > 15cm\n")
    f.write("- Final 15cm bins with deviation > 2° are ignored\n")
    f.write("="*80 + "\n\n")
    
    f.write("ADAPTIVE BIN ANALYSIS (FROM BOTTOM TO TOP):\n")
    f.write("-"*80 + "\n")
    f.write("aplha shape value:1.7\n")
    
    total_waste_points = 0
    total_all_points = 0
    vertical_bins = 0
    tilted_bins = 0
    
    for i, row in enumerate(output_data):
        buffer_id, z_start, z_end, height_from_ground, diameter, radius, height, volume, surface_area, surface_density, inside_points, buffer_zone_points, waste_points, valid_points, equation, deviation_deg, axis_str = row
        
        total_waste_points += waste_points
        total_all_points += (inside_points + buffer_zone_points + waste_points)
        
        if deviation_deg <= max_deviation_deg:
            vertical_bins += 1
            verticality_status = "VERTICAL"
        else:
            tilted_bins += 1
            verticality_status = "TILTED"
        
        waste_percentage = (waste_points / (inside_points + buffer_zone_points + waste_points)) * 100 if (inside_points + buffer_zone_points + waste_points) > 0 else 0
        fit_quality = (valid_points / (inside_points + buffer_zone_points + waste_points)) * 100 if (inside_points + buffer_zone_points + waste_points) > 0 else 0
        
        f.write(f"Bin {buffer_id} - {height_from_ground:.1f}cm from base ({z_start:.1f}-{z_end:.1f} cm) - {verticality_status}:\n")
        f.write(f"  Position: {height_from_ground:.1f}cm above tree base\n")
        f.write(f"  Bin Height: {height:.1f} cm\n")
        f.write(f"  Radius: {radius:.3f} cm\n")
        f.write(f"  Diameter: {diameter:.3f} cm\n")
        f.write(f"  Volume: {volume:.6f} m³\n")
        f.write(f"  Surface Area: {surface_area:.3f} cm²\n")
        f.write(f"  Surface Density: {surface_density:.5f} points/cm²\n")
        f.write(f"  Axis Deviation: {deviation_deg:.2f}° from vertical\n")
        f.write(f"  Axis Vector: {axis_str}\n")
        f.write(f"  Useful Points: {valid_points}\n")
        f.write(f"  Waste Points: {waste_points}\n")
        f.write(f"  Total Points: {inside_points + buffer_zone_points + waste_points}\n")
        f.write(f"  Waste Percentage: {waste_percentage:.1f}%\n")
        f.write(f"  Cylinder Fit Quality: {fit_quality:.1f}%\n")
        f.write(f"  Cylinder Equation: {equation}\n")
        f.write("-"*60 + "\n")
    
    # Add comprehensive summary statistics
    if output_data:
        diameters = [row[4] for row in output_data]  # diameter is at index 4
        deviations = [row[15] for row in output_data]  # deviation is at index 15
        bin_heights = [row[6] for row in output_data]  # height is at index 6
        overall_waste_percentage = (total_waste_points / total_all_points) * 100 if total_all_points > 0 else 0
        
        f.write(f"\nCOMPREHENSIVE SUMMARY STATISTICS:\n")
        f.write(f"="*50 + "\n")
        f.write(f"DIMENSIONAL ANALYSIS:\n")
        f.write(f"  Average diameter: {np.mean(diameters):.3f} cm\n")
        f.write(f"  Base diameter (first bin): {diameters[0]:.3f} cm\n")
        f.write(f"  Top diameter (last bin): {diameters[-1]:.3f} cm\n")
        f.write(f"  Diameter taper: {diameters[0] - diameters[-1]:.3f} cm\n")
        f.write(f"  Taper rate: {(diameters[0] - diameters[-1])/total_height*100:.3f} cm/m\n")
        f.write(f"  Standard deviation of diameters: {np.std(diameters):.3f} cm\n")
        f.write(f"\nBIN SIZE ANALYSIS:\n")
        f.write(f"  Total bins created: {len(output_data)}\n")
        f.write(f"  Average bin height: {np.mean(bin_heights):.1f} cm\n")
        f.write(f"  Minimum bin height: {np.min(bin_heights):.1f} cm\n")
        f.write(f"  Maximum bin height: {np.max(bin_heights):.1f} cm\n")
        f.write(f"  Bins at minimum size ({min_bin_size_cm}cm): {sum(1 for h in bin_heights if abs(h - min_bin_size_cm) < 1)}\n")
        f.write(f"\nVERTICALITY ANALYSIS:\n")
        f.write(f"  Vertical bins (≤ {max_deviation_deg}°): {vertical_bins} ({vertical_bins/len(output_data)*100:.1f}%)\n")
        f.write(f"  Tilted bins (> {max_deviation_deg}°): {tilted_bins} ({tilted_bins/len(output_data)*100:.1f}%)\n")
        f.write(f"  Ignored bins (15cm with > {max_deviation_deg}°): {ignored_bins}\n")
        f.write(f"  Average deviation: {np.mean(deviations):.2f}°\n")
        f.write(f"  Maximum deviation: {np.max(deviations):.2f}°\n")
        f.write(f"  Minimum deviation: {np.min(deviations):.2f}°\n")
        f.write(f"  Standard deviation of axis angles: {np.std(deviations):.2f}°\n")
        f.write(f"\nPOINT DISTRIBUTION:\n")
        f.write(f"  Total waste points: {total_waste_points}\n")
        f.write(f"  Total points analyzed: {total_all_points}\n")
        f.write(f"  Overall waste percentage: {overall_waste_percentage:.1f}%\n")
        f.write(f"  Average points per bin: {total_all_points/len(output_data):.1f}\n")

print("\nAdaptive analysis completed!")
print(f"Created {len(output_data)} final processed bins from adaptive subdivision")
print(f"Ignored {ignored_bins} bins (15cm bins with deviation > {max_deviation_deg}°)")
print(f"Vertical bins (≤ {max_deviation_deg}°): {sum(1 for row in output_data if row[15] <= max_deviation_deg)}")
print(f"Tilted bins (> {max_deviation_deg}°): {sum(1 for row in output_data if row[15] > max_deviation_deg)}")
print("Results saved to 'adaptive_stem_analysis.txt'")
print("\nLegend: Green cylinders = Vertical (≤ 2°), Red cylinders = Tilted (> 2°)")